#### Necessary libraries

- simdjson
    - ```pip install pysimdjson```
- transformers
    - ```pip install transformers```
- pytorch
    - ```pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117```

In [1]:
# load the datasets

import pandas as pd
import simdjson

data_identification = pd.read_csv('dataset/data_identification.csv')
emotion = pd.read_csv('dataset/emotion.csv')
tweets_DM = list[dict]()

json_parser = simdjson.Parser()
with open('dataset/tweets_DM.json', 'r') as f:
    rs = f.read().replace('}\n{', '},{')
    js = json_parser.parse('{"tweets_DM":['+ rs + ']}')
    for t in js['tweets_DM']:
        tweets_DM.append(t['_source']['tweet'].as_dict())

df_tweets_DM = pd.DataFrame(tweets_DM)

In [2]:
# concat the dataframes

df = df_tweets_DM.join(data_identification.set_index('tweet_id'), on='tweet_id').join(emotion.set_index('tweet_id'), on='tweet_id')
df

,hashtags,tweet_id,text,identification,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,NaN
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,NaN
...,...,...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test,NaN
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test,NaN
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,test,NaN
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",train,joy


In [3]:
# identify train and test datasets
# split the train data into 95% train data and 5% data for validation

from sklearn.model_selection import train_test_split

df_train, df_validation = train_test_split(df[df['identification']=='train'], test_size=0.01)
df_test = df[df['identification']=='test']

In [4]:
# EDA

df_train.groupby(['emotion']).count()['text']

emotion
anger            39468
anticipation    246471
disgust         137749
fear             63358
joy             510772
sadness         191516
surprise         48264
trust           203409
Name: text, dtype: int64

In [5]:
# determine the tags from EDA

tags = ['anger', 'anticipation','disgust','fear','joy','sadness','surprise','trust']

In [6]:
# convert into dataset

from datasets import Dataset

ds_train = Dataset.from_pandas(df_train)
ds_validation = Dataset.from_pandas(df_validation)
ds_test = Dataset.from_pandas(df_test)


In [7]:
# map emotions tags with tweet id and drop unnecessary columns

from datasets import ClassLabel

def map_emotion_id(b):
    b['label'] = [cls_labels.str2int(l) if not l is None else None for l in b['emotion']]
    return b

cls_labels = ClassLabel(len(tags),tags)
ds_train = ds_train.map(map_emotion_id, batched=True).cast_column('label', cls_labels).remove_columns(["hashtags", "identification", "emotion", "__index_level_0__"])
ds_validation = ds_validation.map(map_emotion_id, batched=True).cast_column('label', cls_labels).remove_columns(["hashtags", "identification", "emotion", "__index_level_0__"])
ds_test = ds_test.map(map_emotion_id, batched=True).cast_column('label', cls_labels).remove_columns(["hashtags", "identification", "emotion", "__index_level_0__"])

  0%|          | 0/1442 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1442 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/412 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/412 [00:00<?, ?ba/s]

In [8]:
# use the BERT model for tokenize

from transformers import AutoTokenizer

bert = AutoTokenizer.from_pretrained('bert-base-uncased')

In [9]:
# tokenize the datasets

ds_train_tkz = ds_train.map(lambda b: bert(b['text'], truncation=True))
ds_validation_tkz = ds_validation.map(lambda b: bert(b['text'], truncation=True))
ds_test_tkz = ds_test.map(lambda b: bert(b['text'], truncation=True))

  0%|          | 0/1441007 [00:00<?, ?ex/s]

  0%|          | 0/14556 [00:00<?, ?ex/s]

  0%|          | 0/411972 [00:00<?, ?ex/s]

In [10]:
# prepare for pytorch processing

ds_train_tkz.set_format("torch", columns=["input_ids", "attention_mask", "label"])
ds_validation_tkz.set_format("torch", columns=["input_ids", "attention_mask", "label"])
ds_test_tkz.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [11]:
# drop unnecessary columns

ds_train_tkz = ds_train_tkz.remove_columns(['tweet_id'])
ds_validation_tkz = ds_validation_tkz.remove_columns(['tweet_id'])
ds_test_tkz = ds_test_tkz.remove_columns(['label'])

In [12]:
# check whether the graphic card is working

import torch

torch.cuda.is_available()

True

In [14]:
# using the graphic card

from transformers import AutoModelForSequenceClassification

device = torch.device('cuda')

In [24]:
# load the model

model = (AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(tags)).to(device))

loading configuration file config.json from cache at C:\Users\Fia/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidde

In [21]:
# setup the training settings for the model

from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# batch size is actually 256 since I use gradient accumulation
batch_size = 16
logging_steps = len(ds_train_tkz) // batch_size
training_args = TrainingArguments(
        output_dir="results", 
        num_train_epochs=3.0,
        learning_rate=0.00002,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=16,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        weight_decay=0.01,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=50,
        save_steps=500,
        save_total_limit=5,
        group_by_length=True
        )

# use dynamic padding to speed up the training
data_collocator = DataCollatorWithPadding(bert)
trainer = Trainer(
        model=model, 
        args=training_args,
        compute_metrics=lambda pred: {"accuracy": accuracy_score(pred.label_ids, pred.predictions.argmax(-1)), "f1": f1_score(pred.label_ids, pred.predictions.argmax(-1), average="weighted")},
        data_collator=data_collocator,
        train_dataset=ds_train_tkz,
        eval_dataset=ds_validation_tkz
        )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [15]:
# train the model

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\Fia\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1441007
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 16
  Total optimization steps = 16884
  Number of trainable parameters = 109488392


  0%|          | 0/16884 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 14556
  Batch size = 16


  0%|          | 0/910 [00:00<?, ?it/s]

{'eval_loss': 1.631666898727417, 'eval_accuracy': 0.39756801319043694, 'eval_f1': 0.2572647302213501, 'eval_runtime': 36.8309, 'eval_samples_per_second': 395.212, 'eval_steps_per_second': 24.708, 'epoch': 0.01}


...

  0%|          | 0/910 [00:00<?, ?it/s]

{'eval_loss': 0.9290547966957092, 'eval_accuracy': 0.6658422643583402, 'eval_f1': 0.6579696757699764, 'eval_runtime': 38.2878, 'eval_samples_per_second': 380.173, 'eval_steps_per_second': 23.767, 'epoch': 2.99}




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results\checkpoint-16000 (score: 0.6590949018725265).


{'train_runtime': 34918.4423, 'train_samples_per_second': 123.803, 'train_steps_per_second': 0.484, 'train_loss': 0.9465883061152108, 'epoch': 3.0}


TrainOutput(global_step=16884, training_loss=0.9465883061152108, metrics={'train_runtime': 34918.4423, 'train_samples_per_second': 123.803, 'train_steps_per_second': 0.484, 'train_loss': 0.9465883061152108, 'epoch': 3.0})

In [16]:
# save the model so i don't have to train again

bert.save_pretrained('model')
trainer.save_model('model')

tokenizer config file saved in model\tokenizer_config.json
Special tokens file saved in model\special_tokens_map.json
Saving model checkpoint to model
Configuration saved in model\config.json
Model weights saved in model\pytorch_model.bin


In [23]:
# the final prediction result

import numpy as np

prediction = trainer.predict(ds_test_tkz)
prediction_arr = np.argmax(prediction.predictions, axis=1).tolist()
prediction_str = [cls_labels.int2str(l) for l in prediction_arr]
df_test = ds_test_tkz.to_pandas()
df_test['emotion_pred'] = prediction_str
df_test[['tweet_id', 'emotion_pred']].to_csv('pred.csv', index=False, header=['id', 'emotion'])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet_id, text. If tweet_id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 411972
  Batch size = 16


  0%|          | 0/25749 [00:00<?, ?it/s]